<a href="https://colab.research.google.com/github/kuoootina/aop113/blob/main/EX05_03_Reply_Multi_type_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 安裝套件
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 7.0 MB/s eta 0:00:00


In [ ]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
  ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
  ngrok.connect(5000)
  run_with_ngrok(app)


from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent,TextMessageContent
from linebot.v3.messaging import (
    Configuration,ApiClient,MessagingApi,
    ReplyMessageRequest,
    TextMessage, StickerMessage, ImageMessage, VideoMessage, LocationMessage

)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route('/callback', methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    # get request body as text
    body = request.get_data(as_text=True)  # Line平台傳訊息的內容
    app.logger.info("Request body: " + body)
    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)
    return 'OK'


import google.generativeai as genai

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel("gemini-2.0-flash")
def ask_genai(question):
    response = model.generate_content(question)
    return response.resultq


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == 'sticker':
            # 回覆貼圖
            reply = StickerMessage(
               package_id='貼圖包 ID（string）',
               sticker_id='貼圖 ID（string）'
            )
        elif action == 'image':
            # 回覆圖片
            reply = ImageMessage(
               original_content_url='圖片原始網址（高清版）',
               preview_image_url='預覽圖網址（縮圖）'
            )
        elif action == 'video':
            reply = VideoMessage(
              original_content_url='影片的原始影片連結',
              preview_image_url='影片的預覽縮圖連結'
            )
        elif action == 'location':
            reply = LocationMessage(
              title="地點名稱",
              address="地址",
              latitude=緯度,
              longitude=經度
            )

        else:
            response = ask_genai(action)
            reply = TextMessage(text=response)

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    reply
                ]
            )
        )

ngrok_start() # for colab
if __name__ == '__main__':
    app.run()